In [ ]:
import numpy as np

# PenDigits dataset (Variable Length) 

* Can't use ts-learn since it interpolated and homogenized the length of all time series

In [ ]:
#################################################################################################################
## Loading code taken from https://github.com/pafoster/conformance_distance_experiments_cochrane_et_al_2020    ##
## DATASET_URLS = ['https://archive.ics.uci.edu/ml/machine-learning-databases/pendigits/pendigits-orig.tes.Z', ##
##                 'https://archive.ics.uci.edu/ml/machine-learning-databases/pendigits/pendigits-orig.tra.Z'] ##
#################################################################################################################

def read_pendigits_dataset(filename):
    with open(filename, 'r') as f:
        data_lines = f.readlines()

    data = []
    data_labels = []
    current_digit = None
    for line in data_lines:
        if line == "\n":
            continue

        if line[0] == ".":
            if "SEGMENT DIGIT" in line[1:]:
                if current_digit is not None:
                    data.append(np.array(current_digit))
                    data_labels.append(digit_label)

                current_digit = []
                digit_label = int(line.split('"')[1])
            else:
                continue

        else:
            x, y = map(float, line.split())
            current_digit.append([x, y])
            
    data.append(np.array(current_digit))
    data_labels.append(digit_label)
    return data, np.array(data_labels)


def create_pendigits_dataframe(data):
    dataframes = []
    for subset, data in data.items():
        df = pd.DataFrame(data).T
        df.columns = ['data', 'label']
        df['subset'] = subset
        dataframes.append(df)
    return pd.concat(dataframes)

data = {'train': read_pendigits_dataset("Data/pendigits-orig.tra"),
        'test': read_pendigits_dataset("Data/pendigits-orig.tes")}

df_pendigits_raw = create_pendigits_dataframe(data)

def plot_pendigits_entry(sample):
    df = pd.DataFrame(sample["data"], columns=["x", "y"])
    fig = px.line(df, x="x", y="y", text=df.index, width=500, height=500)
    fig.update_traces(textposition="bottom right")
    fig.show()

plot_pendigits_entry(df_pendigits_raw.iloc[20])
print(df_pendigits_raw.head()) 
# Each data point is a timeseries of the form ([x_t1, y_t1], ... , [x_tn, y_tn]) 
# of variable length, that is an array of shape (N_i, 2) for each i in the dataset.


In [ ]:
############################################################################################## |
################################### PenDigits experiments #################################### |
############################################################################################## \/

def run_pendigits_experiments(df:pd.DataFrame, 
                              kernel_names:List[str],
                              stream_transforms = ["time_enhance", "min_max_normalize"],):
    """Calculates AUCs for each kernel on the PenDigits dataset.
    df has columns ["data", "label", "subset"]. Each data point 
    is a timeseries of shape (T_i, d) of variable length."""
    #transform streams
    df["data"] = df["data"].apply(lambda x : transform_stream(x, stream_transforms))

    #Gather dataset info
    X_train = df[df["subset"]=="train"]["data"].values
    y_train = np.array(df[df["subset"]=="train"]["label"].values)
    X_test = df[df["subset"]=="test"]["data"].values
    y_test = np.array(df[df["subset"]=="test"]["label"].values)
    labels = sorted(df["label"].unique())
    num_classes = len(labels)
    d = X_train[0].shape[1]
    T = "variable length"
    N_train = len(X_train)
    N_test = len(X_test)
    print_dataset_stats(num_classes, d, T, N_train, N_test)

    # Run each kernel
    kernel_results = {}
    for kernel_name in kernel_names:
        print(kernel_name)
        scores = run_single_kernel(X_train, y_train, X_test, y_test, labels, 
                        kernel_name, variable_length=True, normalize=False,
                        trunc_sig_dim_bound=200, SVD_max_rank=None)
        kernel_results[kernel_name] = scores

    #log results
    pendigits_results = {"results": kernel_results, 
                         "num_classes": num_classes,
                         "dim": d,
                         "ts_length":T, 
                         "N_train":N_train, 
                         "N_test":N_test}
    return pendigits_results

# pendigits_results = run_pendigits_experiments(
#     df_pendigits_raw, 
#     kernel_names=[
#         #"gak",
#         "truncated signature", 
#         #"signature pde", 
#         #"signature pde RBF"
#         ],
#         )

In [ ]:
# print_experiment_results({"PenDigits": pendigits_results})